This notebook imports the fundamental objects of the streamm.forcefields module and goes through the functionality of each

In [1]:
from pprint import pprint

In [2]:
import logging
logging.basicConfig(filename='forcefield_example.log',level=logging.DEBUG)

In [3]:
import streamm.forcefields.particletype as particletype

Let's start with a ethane molecule from the buildingblocks.ipynb example 

We have an sp3 carbon bonded to hydrogens

Let's create the force field parameters object for a 'CT' carbon and an 'HC' hydrogen

In [4]:
CT = particletype.Particletype('CT')
HC = particletype.Particletype('HC')

Set some parameters from J. Am. Chem. Soc., Vol. 121, No. 20, 1999

In general you should pick a force field that has been shown to work well for your system and set up the parameters 

Check that we have our units set right

In [5]:
print CT.unit_conf['energy'],CT.unit_conf['length']

Ha ang


Our potential is in kCal/mol (`kCalmol`) so let's get the unit dictionary and create our own default

In [6]:
import copy

In [7]:
oplsaa_unit_conf = copy.deepcopy(CT.unit_conf)

In [8]:
oplsaa_unit_conf['energy'] = 'kCalmol'

In [9]:
pprint(oplsaa_unit_conf)

{u'amount': u'atom',
 u'angle': u'degree',
 u'capacitance': u'F',
 u'charge': u'e',
 u'conductance': u'S',
 u'current': u'A',
 u'density': u'amu_nm^3',
 u'electric_dipole_moment': u'D',
 u'emf': u'V',
 u'energy': 'kCalmol',
 u'force': u'GN',
 u'frequency': u'Hz',
 u'harm_bond_coeff': u'kCalmolsqang',
 u'intensity': u'cd',
 u'length': u'ang',
 u'magnetic_flux': u'Wb',
 u'mass': u'amu',
 u'memory': u'Kb',
 u'power': u'GW',
 u'pressure': u'KPa',
 u'resistance': u'ohm',
 u'temperature': u'K',
 u'time': u'ns',
 u'volume': u'nm^3'}


In [10]:
CT.update_units(oplsaa_unit_conf)

In [11]:
HC.update_units(oplsaa_unit_conf)

In [12]:
CT.epsilon = 0.066 # kcal/mol
CT.sigma = 3.5 # Angstroms 

In [13]:
HC.epsilon = 0.03 # kcal/mol
HC.sigma = 2.5 # Angstroms 

Set mass using periodic table

In [14]:
import pymatgen_core.core.periodic_table as periodic_table

In [15]:
CT.mass =  periodic_table.Element['C'].atomic_mass.real
HC.mass =  periodic_table.Element['H'].atomic_mass.real

Set the bond stretching parameters 

In [16]:
import streamm.forcefields.bondtype as bondtype

In [17]:
C_H = bondtype.Bondtype('CT','HC',unit_conf=oplsaa_unit_conf)

In [18]:
C_H.setharmonic(1.080,367.0)

In [19]:
print C_H

 bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  


In [20]:
C_C = bondtype.Bondtype('CT','CT',unit_conf=oplsaa_unit_conf)
C_C.setharmonic(1.53,268.0)

In [21]:
import streamm.forcefields.angletype as angletype

In [22]:
H_C_H = angletype.Angletype('HC','CT','HC',unit_conf=oplsaa_unit_conf)

In [23]:
H_C_H.setharmonic(110.7,37.50)

In [24]:
print H_C_H

 angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  


In [25]:
H_C_C = angletype.Angletype('HC','CT','CT',unit_conf=oplsaa_unit_conf)
H_C_C.setharmonic(110.7,37.50)

Now we need a dihedral potential for the HC-CT-CT-HC dihedral 

In [26]:
import streamm.forcefields.dihtype as dihtype

In [27]:
H_C_C_H = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa_unit_conf)

In [28]:
H_C_C_H.type ='opls'

In [29]:
H_C_C_H.setopls(0.0,0.0,0.3,0.0)

Let's create a parameter container to keep track of our parameters

In [30]:
import streamm.forcefields.parameters as parameters 

In [31]:
paramC = parameters.Parameters('oplsaa',unit_conf=oplsaa_unit_conf)

Add parameters to the container

In [32]:
paramC.add_particletype(CT)

In [33]:
paramC.add_particletype(HC)

In [34]:
paramC.add_bondtype(C_H)
paramC.add_bondtype(C_C)

In [35]:
paramC.add_angletype(H_C_H)
paramC.add_angletype(H_C_C)

In [36]:
paramC.add_dihtype(H_C_C_H)

In [37]:
print paramC


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Imporper Dihedral parameters 0 



In [38]:
for ptkey,pt in paramC.particletypes.iteritems():
    print ptkey,pt,pt.unit_conf['energy'],pt.unit_conf['length']
    

0  CT epsilon:0.066 sigma:3.5 kCalmol ang
1  HC epsilon:0.03 sigma:2.5 kCalmol ang


In [39]:
for btkey,bt in paramC.bondtypes.iteritems():
    print btkey,bt,bt.unit_conf['harm_bond_coeff'],pt.unit_conf['length']

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   kCalmolsqang ang
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0   kCalmolsqang ang


In [40]:
for atkey,at in paramC.angletypes.iteritems():
    print atkey,at,at.unit_conf['energy'],at.unit_conf['length']

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0   kCalmol ang
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0   kCalmol ang


In [41]:
print paramC.tag

oplsaa


In [42]:
paramC.unit_conf

{u'amount': u'atom',
 u'angle': u'degree',
 u'capacitance': u'F',
 u'charge': u'e',
 u'conductance': u'S',
 u'current': u'A',
 u'density': u'amu_nm^3',
 u'electric_dipole_moment': u'D',
 u'emf': u'V',
 u'energy': 'kCalmol',
 u'force': u'GN',
 u'frequency': u'Hz',
 u'harm_bond_coeff': u'kCalmolsqang',
 u'intensity': u'cd',
 u'length': u'ang',
 u'magnetic_flux': u'Wb',
 u'mass': u'amu',
 u'memory': u'Kb',
 u'power': u'GW',
 u'pressure': u'KPa',
 u'resistance': u'ohm',
 u'temperature': u'K',
 u'time': u'ns',
 u'volume': u'nm^3'}

In [43]:
print paramC.dump_pickle()

None


Read in methane .xyz file from the structures example

In [44]:
import streamm.structures.buildingblock as bb

In [45]:
mol_i = bb.Buildingblock('ethane')

In [46]:
mol_i.read_xyz()

In [47]:
print(mol_i.n_particles)

8


Find neighbor list based on bonded radius 

In [48]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Let's set the ffkey for each particle based on the symbol. 

In [49]:
for pk,p in mol_i.particles.iteritems():
    print  p.symbol 
    if( p.symbol == 'C' ):
        p.paramkey = 'CA'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HA' 
    print p.paramkey ,mol_i.bonded_nblist.calc_nnab(pk)


C
CA 4
H
HA 1
H
HA 1
H
HA 1
C
CA 4
H
HA 1
H
HA 1
H
HA 1


This is a bit redundant, but we can think of a more complex molecule where we could use the number of neighbors to write a more complex routine 

Find bonds and bond angles based on neighbor list 

In [50]:
mol_i.bonded_bonds()
mol_i.bonded_angles()
mol_i.bonded_dih()

In [51]:
print mol_i.n_particles

8


Now we can set the particles, bonds, bond angles and dihedrals of the molecule to have parameters

First lets set the particle types 

In [52]:
for pk,p in mol_i.particles.iteritems():
    if( p.paramkey == 'CA' ):
        p.param = CT
        p.param_index = 0
    elif( p.paramkey == 'HA' ):
        p.param = HC
        p.param_index = 1


Now we can set the bond types

In [53]:
for bk,b in mol_i.bonds.iteritems():
    b.param = C_H
    b.param_index = 0 

In [54]:
for ak,a in mol_i.angles.iteritems():
    a.param = H_C_H
    b.param_index = 0 

In [55]:
for dk,d in mol_i.dihedrals.iteritems():
    d.param = H_C_C_H
    d.param_index = 0 

In [56]:
print "Particles "
for pk,p in mol_i.particles.iteritems():
    print p,p.param, p.param_index 
print "\n Bonds "
for bk,b in mol_i.bonds.iteritems():    
    print b,b.param, b.param_index 
print "\n Bond angles "
for ak,a in mol_i.angles.iteritems():
    print a,a.param, a.param_index 
print "\n Dihedrals "
for ak,a in mol_i.dihedrals.iteritems():
    print a,a.param, a.param_index     

Particles 
atom[0] C (C)  CT epsilon:0.066 sigma:3.5 0
atom[1] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[2] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[3] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[4] C (C)  CT epsilon:0.066 sigma:3.5 0
atom[5] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[6] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[7] H (H)  HC epsilon:0.03 sigma:2.5 1

 Bonds 
 0 - 1  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 2  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 3  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 4  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 4 - 5  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 4 - 6  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K

Now our molecule has forcefield parameters for all the interactions 

Now let's say we want to use a software like GROMACS that used kJ/mol instead of kCal/mol

In [57]:
gromacs_unit_conf = copy.deepcopy(oplsaa_unit_conf)

In [58]:
gromacs_unit_conf['energy'] = 'kJmol'
gromacs_unit_conf['length'] = 'nm'

gromacs_unit_conf['harm_bond_coeff'] = 'kJmolsqnm' #*

* The harmonic bond coefficient `harm_bond_coeff` has to be changed as well since it has special units of energy/length^2

In [59]:
pprint(gromacs_unit_conf)

{u'amount': u'atom',
 u'angle': u'degree',
 u'capacitance': u'F',
 u'charge': u'e',
 u'conductance': u'S',
 u'current': u'A',
 u'density': u'amu_nm^3',
 u'electric_dipole_moment': u'D',
 u'emf': u'V',
 u'energy': 'kJmol',
 u'force': u'GN',
 u'frequency': u'Hz',
 u'harm_bond_coeff': 'kJmolsqnm',
 u'intensity': u'cd',
 u'length': 'nm',
 u'magnetic_flux': u'Wb',
 u'mass': u'amu',
 u'memory': u'Kb',
 u'power': u'GW',
 u'pressure': u'KPa',
 u'resistance': u'ohm',
 u'temperature': u'K',
 u'time': u'ns',
 u'volume': u'nm^3'}


In [60]:
mol_i.update_units(gromacs_unit_conf)

In [61]:
print "Particles "
for pk,p in mol_i.particles.iteritems():
    print p,p.param, p.param_index 
print "\n Bonds "
for bk,b in mol_i.bonds.iteritems():    
    print b,b.param, b.param_index 
print "\n Bond angles "
for ak,a in mol_i.angles.iteritems():
    print a,a.param, a.param_index 
print "\n Dihedrals "
for ak,a in mol_i.dihedrals.iteritems():
    print a,a.param, a.param_index      

Particles 
atom[0] C (C)  CT epsilon:0.276144 sigma:0.35 0
atom[1] H (H)  HC epsilon:0.12552 sigma:0.25 1
atom[2] H (H)  HC epsilon:0.12552 sigma:0.25 1
atom[3] H (H)  HC epsilon:0.12552 sigma:0.25 1
atom[4] C (C)  CT epsilon:0.276144 sigma:0.35 0
atom[5] H (H)  HC epsilon:0.12552 sigma:0.25 1
atom[6] H (H)  HC epsilon:0.12552 sigma:0.25 1
atom[7] H (H)  HC epsilon:0.12552 sigma:0.25 1

 Bonds 
 0 - 1  bond  CT - HC type harmonic 
  harmonic r_0 = 0.108000 K = 153552.800000 lammps index 0  gromacs index 0   0
 0 - 2  bond  CT - HC type harmonic 
  harmonic r_0 = 0.108000 K = 153552.800000 lammps index 0  gromacs index 0   0
 0 - 3  bond  CT - HC type harmonic 
  harmonic r_0 = 0.108000 K = 153552.800000 lammps index 0  gromacs index 0   0
 0 - 4  bond  CT - HC type harmonic 
  harmonic r_0 = 0.108000 K = 153552.800000 lammps index 0  gromacs index 0   0
 4 - 5  bond  CT - HC type harmonic 
  harmonic r_0 = 0.108000 K = 153552.800000 lammps index 0  gromacs index 0   0
 4 - 6  bond  CT 

In [62]:
mol_i.update_units(oplsaa_unit_conf)

In [63]:
print "Particles "
for pk,p in mol_i.particles.iteritems():
    print p,p.param, p.param_index 
print "\n Bonds "
for bk,b in mol_i.bonds.iteritems():    
    print b,b.param, b.param_index 
print "\n Bond angles "
for ak,a in mol_i.angles.iteritems():
    print a,a.param, a.param_index 
print "\n Dihedrals "
for ak,a in mol_i.dihedrals.iteritems():
    print a,a.param, a.param_index      

Particles 
atom[0] C (C)  CT epsilon:0.066 sigma:3.5 0
atom[1] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[2] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[3] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[4] C (C)  CT epsilon:0.066 sigma:3.5 0
atom[5] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[6] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[7] H (H)  HC epsilon:0.03 sigma:2.5 1

 Bonds 
 0 - 1  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 2  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 3  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 0 - 4  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 4 - 5  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   0
 4 - 6  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K

Sweet as, bro!